In [1]:
import numpy as np
import scipy as scipy
from scipy.optimize import minimize
import timeit

# Try Solving Many Ancestries problem, multiple SNPs

In this notebook we attempt to solve the following constrained, quadratic optimization problem:

$$\min_{\pi \in \mathbb{R}^k} f(\pi)=\sum_{i=1}^{N}\left(\sum_{j=1}^k a_{j,i}\pi_j-\tilde{a}_i\right)^2$$

$$\text{subject to:} \sum_{j=1}^k \pi_k=1 \quad \pi_j \geq 0, j=1,\ldots,k,$$

where $a_{j,i} \in \mathbb{R}$, $j=1,\ldots, k$; $i=1,\ldots N$ and $\tilde{a}_i \in \mathbb{R}$, $i =1, \ldots, N$ are quantities obtained from a genetics simulation. The $a_{j,i}$'s correspond to the observed allele frequency in ancestry $j$ at SNP $i$. There are $k$ ancestries and $N$ SNPs.

In [101]:
N=50000 # number of SNPs
k=15 # number of ancestries

A=np.array(np.random.uniform(low=0, high=1, size=(N,1))) # initialize an array for experimental draws

for i in range(1,k):
    A=np.hstack((A,np.random.uniform(low=0, high=1, size=(N,1))))

# First, we choose an answer! This vector must be Nx1

ans=[[0.1], [0.15], [0.2], [0.25], [0], [0.05], [0.02], [0.01], [0.005], [0], [0], [0.1], [0.1], [0], [0.005]]

taf=A@ans # Total allele frequency

print(np.shape(A),np.shape(taf), np.shape(ans), np.shape(taf))

(50000, 15) (50000, 1) (15, 1) (50000, 1)


In [102]:
# This is the objective function!

def gen_function(x):
    b=0
    for i in range(0,k):
        b=b + x[i]*A[:,i:(i+1)]
    b=b-taf
    return np.sum(b**2, axis=0)[0]

In [103]:
# This is a feasible initial point since its components add to 1 and are positive.

x_t=(1/k)*np.ones((k,1))
print('check shape of x_t:', np.shape(x_t), np.sum(x_t,axis=0))

# Make sure function works by computing f(x_t)

print('our initial value is', np.transpose(x_t)) # transpose for readability only
print('which has function value', gen_function(x_t))

check shape of x_t: (15, 1) [1.]
our initial value is [[0.06666667 0.06666667 0.06666667 0.06666667 0.06666667 0.06666667
  0.06666667 0.06666667 0.06666667 0.06666667 0.06666667 0.06666667
  0.06666667 0.06666667 0.06666667]]
which has function value 388.3046892151477


In [125]:
# Here is the gradient of the objective function

def gen_gradfun(x):
    
    gradvec = np.zeros((k,1))
    
    d=0
    
    for i in range(0,k):
        d=d + x[i]*A[:,i:(i+1)]
    d=d-taf
    
    for i in range(0,k):
        gradvec[i,:] = np.sum(2*A[:,i:(i+1)]*d, axis=0)
    return gradvec

In [124]:
s=gen_gradfun(x_t)
zero=gen_gradfun(ans)

print('grad of starting point is:',np.transpose(s)) # this is the gradient of where we begin
print('should be zeros:',np.transpose(zero)) # this should be zero if we have the right answer

grad of starting point is: [[  -43.20603307  -467.47809344  -896.14193183 -1297.58718218
    789.54989447   369.59392999   626.05759405   697.61840904
    732.46131016   779.41366605   780.6834401    -47.63155617
    -70.47316651   780.41055318   748.42200588]]
should be zeros: [[-2.82064426e-14 -3.40915396e-14 -4.59081626e-14 -5.51547318e-14
  -3.84164998e-14 -2.25130846e-14 -2.74672093e-14 -2.49143585e-14
  -1.35277338e-14 -2.43695337e-14 -1.46814036e-14 -5.22985240e-15
  -1.32070448e-14 -2.57220122e-14 -2.72564919e-15]]


## SLSQP

In [119]:
# These are wrappers that make our constraints and our bounds

cons = ({'type': 'eq', 'fun': lambda x:  np.sum(x,axis=0) -1},)

for i in range(0,k-1):
    cons = cons + ({'type': 'ineq', 'fun': lambda x: x[i]},)

bnds = ((0, None),)

for i in range(0,k-1):
    bnds = bnds + ((0, None),)

In [120]:
# This cell runs and times SLSQP

start = timeit.default_timer()

print(scipy.optimize.minimize(gen_function, x_t, method='SLSQP', jac=gen_gradfun, bounds=bnds, constraints=cons, tol=1e-5))

stop = timeit.default_timer()

print('Time: ', stop - start)

print('our correct answer was chosen to be', ans)

     fun: 1.2760067618512503
     jac: array([254.6406695 , 255.16149974, 255.30448387, 254.83016562,
       255.81706791, 254.99902426, 254.91936865, 254.78982926,
       254.96373357, 255.67967757, 255.44170472, 255.04227613,
       255.04867348, 256.00006087, 254.85080864])
 message: 'Optimization terminated successfully.'
    nfev: 68
     nit: 29
    njev: 29
  status: 0
 success: True
       x: array([0.10071637, 0.15068574, 0.20064008, 0.25051084, 0.00070234,
       0.05056845, 0.02069921, 0.01067863, 0.00557056, 0.00072031,
       0.00078654, 0.1005591 , 0.10068855, 0.00088922, 0.00558407])
Time:  1.2299670490710923
our correct answer was chosen to be [[0.1], [0.15], [0.2], [0.25], [0], [0.05], [0.02], [0.01], [0.005], [0], [0], [0.1], [0.1], [0], [0.005]]


In [116]:
# How do we call the computed answer, x, without copy/pasting?!

x=np.array([0.10071637, 0.15068574, 0.20064008, 0.25051084, 0.00070234,
       0.05056845, 0.02069921, 0.01067863, 0.00557056, 0.00072031,
       0.00078654, 0.1005591 , 0.10068855, 0.00088922, 0.00558407])

In [117]:
# Print out the error in the worst component

np.max(abs(x-np.transpose(ans)))

0.00088922

In [118]:
# Print out average error

(1/k)*np.sum(abs(x-np.transpose(ans)),axis=0)[0]

4.77579999999996e-05